This code performs Virtual screening process

protein 2BQV already cleaned , removed HOH , defined corordinats already having ligand using pymol and performed site directed docking using vina in colab

Approximately 500 ligands downloaded as dataset from pubchem using filter as "fda approved drugs" and saved as csv format.

In [1]:
# Install autodock vina
!apt-get update
!apt-get install -y autodock-vina
# Install rdkit
!pip install rdkit
# Install openbabel
!pip install openbabel-wheel biopython

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,008 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,802 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,274 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-dri

In [2]:
from google.colab import files

# Upload receptor.pdbqt
uploaded = files.upload()

Saving 2BQV_prepred.pdbqt to 2BQV_prepred.pdbqt


In [4]:
# Load SMILES file [Drug datset obtained from pubchem]
import pandas as pd
df = pd.read_csv("/content/pubchem_fda_subset_500_cleaned.csv")
df.head()


,smiles,compound_id,MolWt,LogP,NumHDonors,NumHAcceptors
0,CN1CCN(C2=Nc3cc(Cl)ccc3Nc3ccccc32)CC1,135398737,326.831,3.7227,1.0,4.0
1,CC(=O)Oc1ccc(C(c2ccc(OC(C)=O)cc2)c2ccccn2)cc1,2391,361.397,4.1124,0.0,5.0
2,CN1CCC[C@H]1c1cccnc1,89594,162.236,1.8483,0.0,2.0
3,CN(C)CCC=C1c2ccccc2CCc2ccccc21,2160,277.411,4.1686,0.0,1.0
4,CN[C@H](C)Cc1ccc2c(c1)OCO2,1615,193.246,1.5657,1.0,3.0


In [5]:
## Use RDKit for 3D Generation

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools
import os

In [6]:
import os
from rdkit import Chem
from rdkit.Chem import AllChem

# Make output folder
os.makedirs("ligands_pdb", exist_ok=True)

for i, row in df.iterrows():
    smiles = row['smiles']      # column in your CSV
    cid = str(row['compound_id'])  # use compound_id as filename
    mol = Chem.MolFromSmiles(smiles)

    if mol:
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol, AllChem.ETKDG())
        AllChem.UFFOptimizeMolecule(mol)
        out_file = f"ligands_pdb/{cid}.pdb"
        Chem.MolToPDBFile(mol, out_file)

print("✅ Finished generating PDB files for all ligands.")

✅ Finished generating PDB files for all ligands.


In [7]:
## convert the PDBs to PDBQT for docking:
# will create the folder ligands_pdbqt for your .pdbqt ligands.
import os

os.makedirs("ligands_pdbqt", exist_ok=True)

In [8]:
# run OpenBabel to convert your RDKit-generated PDB files → PDBQT:

!obabel ligands_pdb/*.pdb -O ligands_pdbqt/ligand_.pdbqt -m --partialcharge gasteiger

422 molecules converted
422 files output. The first is ligands_pdbqt/ligand_1004.pdbqt


In [ ]:
##  422 instead of 500, because some SMILES failed to generate valid 3D coordinates

In [ ]:
## Next Step: Set Up Docking with AutoDock Vina

	# 1.	Receptor: receptor.pdbqt  already prepared 2BQV).
	# 2.	Ligands: ligands_pdbqt/*.pdbqt (422 ligands).
	# 3.	Define the docking grid box (around AHA455 ligand from 2BQV):

In [ ]:
# ## Define the docking grid box (around AHA455 ligand from 2BQV):

# # From pymol we got grid center as

# min: [7.727, 18.230, 1.791]
# max: [19.998, 28.430, 10.025]

#  Step 1: Calculate Grid Center

# Grid center = average of min and max for each axis:

# center_x = (7.727 + 19.998)/2 ≈ 13.863
# center_y = (18.230 + 28.430)/2 ≈ 23.330
# center_z = (1.791 + 10.025)/2 ≈ 5.908

#  Step 2: Calculate Grid Size

# Grid size = (max - min) + buffer (~5 Å):

# size_x = (19.998 - 7.727) + 5 ≈ 17.271
# size_y = (28.430 - 18.230) + 5 ≈ 15.200
# size_z = (10.025 - 1.791) + 5 ≈ 13.234


# round them to nearest whole number for simplicity:

# size_x = 17
# size_y = 15
# size_z = 13

In [9]:
###### Just sample only one ligand docking [SITE SPECIFIC]... ######

!vina --receptor 2BQV_prepred.pdbqt \
      --ligand ligands_pdbqt/ligand_1004.pdbqt \
      --center_x 14 --center_y 23 --center_z 5 \
      --size_x 17 --size_y 15 --size_z 13 \
      --out results/ligand_1004_out.pdbqt \
      --exhaustiveness 8 \
      --num_modes 9

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [10]:
import os
import pandas as pd

# Create results folder
os.makedirs("results", exist_ok=True)

# List all ligand files
ligands = sorted(os.listdir("ligands_pdbqt"))

# Select only first 100 ligands
ligands_to_dock = ligands[:100] ### can select for all 422 also

# Prepare dataframe to save results
docking_results = []

# Loop through selected ligands
for ligand in ligands_to_dock:
    ligand_path = os.path.join("ligands_pdbqt", ligand)
    out_path = os.path.join("results", ligand.replace(".pdbqt", "_out.pdbqt"))

    # Run AutoDock Vina
    cmd = f"vina --receptor 2BQV_prepred.pdbqt --ligand {ligand_path} " \
          f"--center_x 13.863 --center_y 23.330 --center_z 5.908 " \
          f"--size_x 17.271 --size_y 15.200 --size_z 13.234 " \
          f"--out {out_path} --exhaustiveness 8 --num_modes 9"
    print(f"Docking {ligand} ...")
    os.system(cmd)

    # Extract best affinity from the output
    best_affinity = None
    with open(out_path) as f:
        for line in f:
            if line.startswith("REMARK VINA RESULT"):
                best_affinity = float(line.split()[3])  # affinity in kcal/mol
                break
    docking_results.append({"ligand": ligand, "best_affinity": best_affinity})

# Save results to CSV
df_results = pd.DataFrame(docking_results)
df_results.to_csv("results/docking_summary.csv", index=False)
print("Docking of first 100 ligands complete! Summary saved to results/docking_summary.csv")

Docking ligand_1004.pdbqt ...
Docking ligand_1045.pdbqt ...
Docking ligand_1046.pdbqt ...
Docking ligand_10607.pdbqt ...
Docking ligand_1066.pdbqt ...
Docking ligand_1102.pdbqt ...
Docking ligand_1110.pdbqt ...
Docking ligand_1118.pdbqt ...
Docking ligand_11333.pdbqt ...
Docking ligand_1176.pdbqt ...
Docking ligand_119.pdbqt ...
Docking ligand_12035.pdbqt ...
Docking ligand_126941.pdbqt ...
Docking ligand_1349907.pdbqt ...
Docking ligand_135398513.pdbqt ...
Docking ligand_135398658.pdbqt ...
Docking ligand_135398737.pdbqt ...
Docking ligand_135401907.pdbqt ...
Docking ligand_14052.pdbqt ...
Docking ligand_1548887.pdbqt ...
Docking ligand_1548943.pdbqt ...
Docking ligand_15541.pdbqt ...
Docking ligand_1614.pdbqt ...
Docking ligand_1615.pdbqt ...
Docking ligand_16362.pdbqt ...
Docking ligand_1645.pdbqt ...
Docking ligand_1646.pdbqt ...
Docking ligand_1727.pdbqt ...
Docking ligand_176.pdbqt ...
Docking ligand_1775.pdbqt ...
Docking ligand_18381.pdbqt ...
Docking ligand_187.pdbqt ...
Docki

In [11]:
#### VIRTUAL SCREENING PROCESS #####

import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

# Load docking summary
docking_df = pd.read_csv("results/docking_summary.csv")

In [12]:
# Load ligand info from PubChem CSV
pubchem_df = pd.read_csv("/content/pubchem_fda_subset_500_cleaned.csv")

In [13]:
# Merge on compound_id
docking_df['compound_id'] = docking_df['ligand'].str.extract(r'(\d+)').astype(int)

In [14]:
# Merge docking results with PubChem data
merged_df = pd.merge(docking_df, pubchem_df, on="compound_id")

In [15]:
# Rank by docking score (more negative = better)
merged_df = merged_df.sort_values("best_affinity")

In [16]:
# Apply Lipinski's Rule of 5
lipinski_df = merged_df[
    (merged_df['MolWt'] <= 500) &
    (merged_df['LogP'] <= 5) &
    (merged_df['NumHDonors'] <= 5) &
    (merged_df['NumHAcceptors'] <= 10)
]

In [17]:
# Save top hits
lipinski_df.to_csv("results/top_hits.csv", index=False)
print("Virtual screening complete! Top hits saved to results/top_hits.csv")

Virtual screening complete! Top hits saved to results/top_hits.csv


In [18]:
print(lipinski_df)

                    ligand  best_affinity  compound_id  \
16  ligand_135398737.pdbqt         -9.112    135398737   
80       ligand_2391.pdbqt         -9.057         2391   
72       ligand_2353.pdbqt         -9.039         2353   
98       ligand_2662.pdbqt         -9.031         2662   
12     ligand_126941.pdbqt         -8.995       126941   
..                     ...            ...          ...   
9        ligand_1176.pdbqt         -2.846         1176   
29        ligand_176.pdbqt         -2.839          176   
57        ligand_222.pdbqt         -1.410          222   
86      ligand_24841.pdbqt         -0.881        24841   
97        ligand_260.pdbqt         -0.879          260   

                                               smiles    MolWt     LogP  \
16              CN1CCN(C2=Nc3cc(Cl)ccc3Nc3ccccc32)CC1  326.831  3.72270   
80      CC(=O)Oc1ccc(C(c2ccc(OC(C)=O)cc2)c2ccccn2)cc1  361.397  4.11240   
72        COc1ccc2cc3[n+](cc2c1OC)CCc1cc2c(cc1-3)OCO2  336.367  3.09630   
98 